In [1]:
!pip install pyswarms
!pip install scikit-fuzzy
!pip install imblearn


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),          # Convert images to tensors
    transforms.Normalize((0.5,), (0.5,))  # Normalize images
])

# Load dataset
dataset_path = "input"
dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

# Create DataLoader
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Check class mapping
print(dataset.class_to_idx)  # {'1': 0, '2': 1, '3': 2, '4': 3}

# Example: Load a batch
for images, labels in dataloader:
    print(f"Batch size: {images.shape}, Labels: {labels}")
    break


{'Mild Dementia': 0, 'Moderate Dementia': 1, 'Non Demented': 2, 'Very mild Dementia': 3}
Batch size: torch.Size([32, 3, 224, 224]), Labels: tensor([2, 0, 1, 3, 3, 0, 3, 3, 0, 2, 0, 3, 0, 0, 0, 0, 3, 0, 0, 3, 3, 1, 3, 3,
        2, 2, 0, 1, 2, 0, 0, 0])


In [2]:
import os
import shutil
import random
from PIL import Image
import numpy as np

# Set paths
dataset_path = "input"  # Change this to your dataset path
output_dir = "Processed_output_Dataset"  # Output directory

# Define train-test split ratio
train_ratio = 0.8

# Target image size
img_size = (224, 224)

# Create train & test directories
train_dir = os.path.join(output_dir, "train")
test_dir = os.path.join(output_dir, "test")

for folder in ["Mild Dementia", "Moderate Dementia","Non Demented","Very mild Dementia"]:  # Loop through severity levels
    os.makedirs(os.path.join(train_dir, folder), exist_ok=True)
    os.makedirs(os.path.join(test_dir, folder), exist_ok=True)

# Function to preprocess and save images
def process_and_split_images():
    for category in ["Mild Dementia", "Moderate Dementia","Non Demented","Very mild Dementia"]:  # Iterate through class folders
        class_path = os.path.join(dataset_path, category)
        images = os.listdir(class_path)
        random.shuffle(images)  # Shuffle for randomness

        split_index = int(len(images) * train_ratio)
        train_images, test_images = images[:split_index], images[split_index:]

        for image_name in train_images:
            process_and_save(image_name, class_path, os.path.join(train_dir, category))

        for image_name in test_images:
            process_and_save(image_name, class_path, os.path.join(test_dir, category))

# Function to load, preprocess, and save images
def process_and_save(image_name, src_folder, dest_folder):
    src_path = os.path.join(src_folder, image_name)
    dest_path = os.path.join(dest_folder, image_name)

    try:
        with Image.open(src_path) as img:
            img = img.convert("RGB")  # Convert to RGB (if grayscale)
            img = img.resize(img_size)  # Resize image
            img.save(dest_path)  # Save processed image
    except Exception as e:
        print(f"Error processing {src_path}: {e}")

# Run preprocessing
process_and_split_images()
print(" Dataset Preprocessing & Splitting Completed!")

 Dataset Preprocessing & Splitting Completed!


In [3]:
import csv
import os
from collections import Counter

# Define CSV file path
csv_file = os.path.join(output_dir, "oasis_dataset_info.csv")

# Collect image data
image_data = []
labels = []
for split in ["train", "test"]: 
    split_dir = os.path.join("input", split)
    split_dir = os.path.join(output_dir, split)
    for category in ["Mild Dementia", "Moderate Dementia","Non Demented","Very mild Dementia"]:
        category_dir = os.path.join(split_dir, category)
        for image_name in os.listdir(category_dir):
            image_path = os.path.join(category_dir, image_name)
            image_data.append([image_path, image_name, category])
            labels.append(category)

# Write to CSV file
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Image_Path", "Image_Name", "Label"])
    writer.writerows(image_data)

print(f"CSV file saved at {csv_file}")

CSV file saved at Processed_output_Dataset\oasis_dataset_info.csv


In [5]:
import pandas as pd
df=pd.read_csv("Processed_output_Dataset/oasis_dataset_info.csv")

In [6]:
# View the first few rows
print(df.head())

# Check basic information
print(df.info())

# Check the label distribution
print(df['Label'].value_counts())


                                          Image_Path  \
0  Processed_output_Dataset\train\Mild Dementia\O...   
1  Processed_output_Dataset\train\Mild Dementia\O...   
2  Processed_output_Dataset\train\Mild Dementia\O...   
3  Processed_output_Dataset\train\Mild Dementia\O...   
4  Processed_output_Dataset\train\Mild Dementia\O...   

                    Image_Name          Label  
0  OAS1_0028_MR1_mpr-1_100.jpg  Mild Dementia  
1  OAS1_0028_MR1_mpr-1_101.jpg  Mild Dementia  
2  OAS1_0028_MR1_mpr-1_102.jpg  Mild Dementia  
3  OAS1_0028_MR1_mpr-1_103.jpg  Mild Dementia  
4  OAS1_0028_MR1_mpr-1_104.jpg  Mild Dementia  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9488 entries, 0 to 9487
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Image_Path  9488 non-null   object
 1   Image_Name  9488 non-null   object
 2   Label       9488 non-null   object
dtypes: object(3)
memory usage: 222.5+ KB
None
Label
Mild Dementia   

In [8]:
from collections import Counter
# Calculate class weights
label_counts = Counter(labels)
total_samples = sum(label_counts.values())
class_weights = {label: total_samples / (len(label_counts) * count) for label, count in label_counts.items()}

print("Class Weights:", class_weights)


Class Weights: {'Mild Dementia': 0.7906666666666666, 'Moderate Dementia': 4.860655737704918, 'Non Demented': 0.7906666666666666, 'Very mild Dementia': 0.7906666666666666}


Summary of Steps
Use descriptive statistics to understand the dataset.
Plot label distributions and pixel intensity histograms.
Visualize class imbalances to determine the need for class weights.
Explore correlations (if applicable).
Check image augmentation and ensure consistency in preprocessing.

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

# Swish activation function
class Swish(nn.Module):
    def forward(self, x):
        return x * torch.sigmoid(x)

# Define the model with Swish activation
class SwishResNet(nn.Module):
    def __init__(self, num_classes=4):
        super(SwishResNet, self).__init__()
        # Load pre-trained ResNet model
        self.resnet = models.resnet18(pretrained=True)
        # Replace ReLU with Swish in the model
        self.resnet.relu = Swish()
        # Modify the final layer to match the number of classes
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, num_classes)

    def forward(self, x):
        return self.resnet(x)

# Instantiate the model
model = SwishResNet()

# Set up optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()


c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [14]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision import models
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import numpy as np
import random

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define Swish activation function
class Swish(nn.Module):
    def forward(self, x):
        return x * torch.sigmoid(x)

# Define a CNN model with Swish activation
class SwishResNet(nn.Module):
    def __init__(self, num_classes=4):
        super(SwishResNet, self).__init__()
        self.resnet = models.resnet50(pretrained=True)
        self.resnet.relu = Swish()  # Replace ReLU with Swish
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, num_classes)
    
    def forward(self, x):
        return self.resnet(x)

from sklearn.preprocessing import LabelEncoder

# Custom Dataset Class
class ImageDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        df = pd.read_csv(csv_file)
        self.transform = transform
        self.data = df.groupby("Label").head(10).reset_index(drop=True)

        # Encode string labels to numeric
        self.label_encoder = LabelEncoder()
        self.data["EncodedLabel"] = self.label_encoder.fit_transform(self.data["Label"])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data.iloc[idx, 0]
        label = torch.tensor(self.data.iloc[idx]["EncodedLabel"], dtype=torch.long)  # <-- FIXED
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label



# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load dataset
csv_file = "Processed_output_Dataset/oasis_dataset_info.csv"
dataset = ImageDataset(csv_file, transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

# Mayfly Algorithm for hyperparameter tuning
def mayfly_optimization(num_mayflies=5, generations=1):
    best_lr, best_wd = 0.0001, 0.0005  # Default values
    best_acc = 0.0

    mayflies = [(random.uniform(1e-5, 1e-3), random.uniform(1e-6, 1e-3)) for _ in range(num_mayflies)]
    
    for gen in range(generations):
        for i, (lr, wd) in enumerate(mayflies):
            model = SwishResNet(num_classes=4).to(device)
            optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
            criterion = nn.CrossEntropyLoss()
            
            model.train()
            correct, total = 0, 0
            for images, labels in train_loader:
                images, labels = images.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
            
            acc = 100 * correct / total
            if acc > best_acc:
                best_acc = acc
                best_lr, best_wd = lr, wd
            
            # Ensure learning rate and weight decay are positive and adaptive
            mayflies[i] = (
                max(1e-5, best_lr * (1 + random.uniform(-0.1, 0.1))),  # Adjust LR in small steps
                max(1e-6, best_wd * (1 + random.uniform(-0.1, 0.1)))   # Adjust WD adaptively
            )
        
        print(f"Generation {gen+1}: , Best LR = {best_lr:.6f}, Best WD = {best_wd:.6f}")
    
    return best_lr, best_wd

# Find best hyperparameters
best_lr, best_wd = mayfly_optimization()

# Train final model with optimized hyperparameters
model = SwishResNet(num_classes=4).to(device)
optimizer = optim.Adam(model.parameters(), lr=best_lr, weight_decay=best_wd)
criterion = nn.CrossEntropyLoss()

def train_model(model, train_loader, criterion, optimizer, epochs=15):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        correct, total = 0, 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        accuracy = 100-correct / total
        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss:.4f}, Accuracy: {accuracy:.2f}%")

def test_model(model, test_loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = 100 -correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")

# Train and test with optimized parameters
train_model(model, train_loader, criterion, optimizer, epochs=2)
test_model(model, test_loader)


c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Generation 1: , Best LR = 0.000639, Best WD = 0.000305
Epoch 1/2, Loss: 10.6865, Accuracy: 99.56%
Epoch 2/2, Loss: 8.9619, Accuracy: 99.41%
Test Accuracy: 99.88%


****

In [15]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
# **Fuzzy Decision-Based Classification**
def fuzzy_decision(output_probs, threshold=0.6):
    """
    Apply fuzzy logic to classify an image.
    If the highest probability is below a threshold, classify as 'uncertain'.
    """
    max_prob, pred_class = torch.max(output_probs, dim=1)  # Get highest probability and class index
    pred_class[max_prob < threshold] = -1  # If probability is low, mark as 'uncertain'
    return pred_class.cpu().numpy()

# Testing function with fuzzy decision logic
def test_model(model, test_loader):
    model.eval()
    true_labels = []
    predicted_labels = []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            probs = torch.nn.functional.softmax(outputs, dim=1)  # Convert to probabilities
            preds = fuzzy_decision(probs)  # Apply fuzzy decision logic

            true_labels.extend(labels.cpu().numpy())
            predicted_labels.extend(preds)

    return true_labels, predicted_labels

# Train the model
train_model(model, train_loader, criterion, optimizer, epochs=20)

# Evaluate the model
y_true, y_pred = test_model(model, test_loader)

# Convert -1 to "Uncertain" class
y_pred = np.array(y_pred)
y_pred_labels = np.where(y_pred == -1, "Uncertain", y_pred)
# Convert all true and predicted labels to strings
y_true_str = [str(label) for label in y_true]
y_pred_str = [str(label) for label in y_pred_labels]

# Now, generate the classification report
print("\nClassification Report:\n")
print(classification_report(y_true_str, y_pred_str, zero_division=1))




Epoch 1/20, Loss: 9.0801, Accuracy: 99.47%
Epoch 2/20, Loss: 5.5937, Accuracy: 99.25%
Epoch 3/20, Loss: 6.1902, Accuracy: 99.25%
Epoch 4/20, Loss: 5.5805, Accuracy: 99.34%
Epoch 5/20, Loss: 3.1035, Accuracy: 99.09%
Epoch 6/20, Loss: 2.8946, Accuracy: 99.12%
Epoch 7/20, Loss: 2.7700, Accuracy: 99.06%
Epoch 8/20, Loss: 1.5184, Accuracy: 99.03%
Epoch 9/20, Loss: 4.3754, Accuracy: 99.12%
Epoch 10/20, Loss: 1.9333, Accuracy: 99.06%
Epoch 11/20, Loss: 0.8503, Accuracy: 99.03%
Epoch 12/20, Loss: 1.5541, Accuracy: 99.09%
Epoch 13/20, Loss: 2.2333, Accuracy: 99.03%
Epoch 14/20, Loss: 1.1694, Accuracy: 99.06%
Epoch 15/20, Loss: 0.7322, Accuracy: 99.03%
Epoch 16/20, Loss: 5.4318, Accuracy: 99.22%
Epoch 17/20, Loss: 2.8081, Accuracy: 99.12%
Epoch 18/20, Loss: 5.2534, Accuracy: 99.16%
Epoch 19/20, Loss: 4.2895, Accuracy: 99.16%
Epoch 20/20, Loss: 1.5259, Accuracy: 99.03%

Classification Report:

              precision    recall  f1-score   support

           0       1.00      0.50      0.67      